<style>
#notebook-container .rendered_html table {
    display: none;
}
</style>

# <div style="padding:15px;background-color:#f4ebdc;border-radius:20px;margin:0;color:#8b4726;border:2px dotted #aaaaaa;font-family:Charter;font-weight: bold;font-size:130%;text-align:center;overflow:hidden;font-weight:500">AI Stock Forecast 📈 Using LSTMs</div>

Welcome to my notebook on AI Stock Forecast 📈 Using LSTMs! In this project, we will be using a dataset called C3.AI Stocks Dataset, which is a collection of stock market data sourced from Yahoo for the period from March 2022 to March 2023. This dataset contains several columns of data for each stock, including the date on which the stock market data was recorded, the opening price of the stock, the highest and lowest prices of the stock on a given day, the closing price of the stock, the adjusted closing price that accounts for any corporate actions that may have affected the stock's price, and the total number of shares traded in the stock on a given day.

Our goal in this project is to use Long Short-Term Memory (LSTM) neural networks, a type of Recurrent Neural Networks (RNN), to predict the future stock prices of the AI company. LSTM networks are a powerful type of neural networks that can capture long-term dependencies in time-series data, making them ideal for stock price prediction tasks.

We will start by performing exploratory data analysis on the C3.AI Stocks Dataset to gain insights into the data and understand the trends and patterns. Then, we will preprocess the data and split it into training, validation and testing sets. After that, we will build and train our LSTM model on the training set and use it to predict the future stock prices on other sets.

By the end of this notebook, you will have a solid understanding of how to use LSTM networks to forecast stock prices and will be able to apply this knowledge to other forecasting tasks as well. Let's get started!

# <div style="color:white;display:fill;border-radius:20px;background-color:#f4ebdc;font-size:70%;font-family:Charter;letter-spacing:1px"><h3 style='padding: 20px;color:#8b4726;text-align:center;'>Importing Main Dependencies</h3></div>

In [1]:
import pandas as pd # for data manipulation
import numpy as np # for linear algebra
import matplotlib.pyplot as plt # for plotting
import plotly.express as px # for plotting
import plotly.graph_objs as go # for plotting

# <div style="color:white;display:fill;border-radius:20px;background-color:#f4ebdc;font-size:70%;font-family:Charter;letter-spacing:1px"><h3 style='padding: 20px;color:#8b4726;text-align:center;'>Loading Dataset</h3></div>

In [2]:
# Getting path of dataset
dataset_path = "/kaggle/input/c3ai-stocks-dataset/AI.csv"

In [3]:
# Loading dataset
df = pd.read_csv(dataset_path, 
                 usecols=['Date','Close'], 
                 parse_dates=['Date'], 
                 index_col='Date')

# Displaying our data
df.head()

,Close
Date,
2022-03-10,19.959999
2022-03-11,18.000000
2022-03-14,16.910000
2022-03-15,17.469999
2022-03-16,19.299999


For simplicity we are gonna use only 'Close' column.

# <div style="color:white;display:fill;border-radius:20px;background-color:#f4ebdc;font-size:70%;font-family:Charter;letter-spacing:1px"><h3 style='padding: 20px;color:#8b4726;text-align:center;'>Exploratory Data Analysis</h3></div>

In [4]:
# Printing shape of dataframe
df.shape

(251, 1)

Data from 1 year has only 251 records which make sense.

In [5]:
# Checking data types of features
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 251 entries, 2022-03-10 to 2023-03-09
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   251 non-null    float64
dtypes: float64(1)
memory usage: 3.9 KB


Data types are ok.

In [6]:
# Checking number of missing values
df.isna().sum()

Close    0
dtype: int64

There are no missing values.

In [7]:
# Plotting price of stock over the months
px.line(df, x=df.index, y='Close', title='Stock Price Over The Months', labels={'x':'Date', 'y':'Price'})

From plot we can se that in last months the price was going high pretty fast

# <div style="color:white;display:fill;border-radius:20px;background-color:#f4ebdc;font-size:70%;font-family:Charter;letter-spacing:1px"><h3 style='padding: 20px;color:#8b4726;text-align:center;'>Preprocessing</h3></div>

### Feature Extraction

As preprocessing operation we age going to use feature extraction to create new lag features using shift method from pandas. <br>
Feature extraction is a process of creating new features as it allows us to engineer new features that may better capture patterns and relationships in the data.

In [8]:
# Creating Lag features
df['lag1'] = df['Close'].shift(1)
df['lag2'] = df['Close'].shift(2)
df['lag3'] = df['Close'].shift(3)

# Displaying our data
df.head()

,Close,lag1,lag2,lag3
Date,,,,
2022-03-10,19.959999,NaN,NaN,NaN
2022-03-11,18.000000,19.959999,NaN,NaN
2022-03-14,16.910000,18.000000,19.959999,NaN
2022-03-15,17.469999,16.910000,18.000000,19.959999
2022-03-16,19.299999,17.469999,16.910000,18.000000


As we can see, the feature creation process was successful in generating new lag features.<br>
However, some rows now contain NaN values due to a lack of past data.<br>
To ensure the dataset is ready for analysis, we will need to drop these rows.

In [9]:
# Dropping features that
df = df.dropna()

Now the data doesn't contain inconsistencies.

### Splitting Data

Splitting data into X and y and later into train, val and test sets.

In [10]:
# Chosing target feature
target = "Close"

# Splitting data into X and y
X = df.drop(columns=[target])
y = df[target]
dates = df.index

# Printing X
X.head()

,lag1,lag2,lag3
Date,,,
2022-03-15,16.910000,18.000000,19.959999
2022-03-16,17.469999,16.910000,18.000000
2022-03-17,19.299999,17.469999,16.910000
2022-03-18,20.750000,19.299999,17.469999
2022-03-21,21.959999,20.750000,19.299999


Data has been split into X and y.

In [11]:
# Converting data to numerical format
X = X.to_numpy().astype(np.float32).reshape(X.shape[0], -1, 1)
y = y.to_numpy().astype(np.float32)
dates = dates.to_numpy()

print(f"dates shape = {dates.shape},\nX shape = {X.shape},\ny shape = {y.shape}")

dates shape = (248,),
X shape = (248, 3, 1),
y shape = (248,)


Data has correct format.\
Now we are going to split it into train, validation and test sets.

In [12]:
# Train/Val/Test split
q_80 = int(len(dates) * .70)
q_90 = int(len(dates) * .90)

dates_train, X_train, y_train = dates[:q_80], X[:q_80], y[:q_80]
dates_val, X_val, y_val = dates[q_80:q_90], X[q_80:q_90], y[q_80:q_90]
dates_test, X_test, y_test = dates[q_90:], X[q_90:], y[q_90:]

### Plotting Sets

In [13]:
# Creating traces for each dataset
trace_train = go.Scatter(x=dates_train, y=y_train, name='Train')
trace_val = go.Scatter(x=dates_val, y=y_val, name='Validation')
trace_test = go.Scatter(x=dates_test, y=y_test, name='Test')

# Combining traces, setting layout and plotting figure
data = [trace_train, trace_val, trace_test]
layout = go.Layout(title='Datasets Plot')
fig = go.Figure(data=data, layout=layout)
fig.show()

As we can see data is correctly spitted.

# <div style="color:white;display:fill;border-radius:20px;background-color:#f4ebdc;font-size:70%;font-family:Charter;letter-spacing:1px"><h3 style='padding: 20px;color:#8b4726;text-align:center;'>Building LSTM Model</h3></div>

Step in which we are going to build and train LSTM model using Keras library.

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers

model = Sequential([layers.Input((3, 1)),
                    layers.LSTM(32),
                    layers.Dense(16, activation='relu'),
                    layers.Dense(16, activation='relu'),
                    layers.Dense(1)])

model.compile(loss='mse', 
              optimizer=Adam(learning_rate=0.003),
              metrics=['mean_absolute_error'])

model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=80)

Epoch 1/80
6/6 [==============================] - 3s 112ms/step - loss: 301.4893 - mean_absolute_error: 17.0142 - val_loss: 143.7497 - val_mean_absolute_error: 11.7874
Epoch 2/80
6/6 [==============================] - 0s 9ms/step - loss: 267.8904 - mean_absolute_error: 15.9847 - val_loss: 110.1572 - val_mean_absolute_error: 10.2563
Epoch 3/80
6/6 [==============================] - 0s 10ms/step - loss: 216.8987 - mean_absolute_error: 14.2697 - val_loss: 64.1041 - val_mean_absolute_error: 7.6762
Epoch 4/80
6/6 [==============================] - 0s 9ms/step - loss: 144.5259 - mean_absolute_error: 11.4172 - val_loss: 17.6292 - val_mean_absolute_error: 3.4952
Epoch 5/80
6/6 [==============================] - 0s 9ms/step - loss: 62.6059 - mean_absolute_error: 6.8143 - val_loss: 10.3875 - val_mean_absolute_error: 2.9476
Epoch 6/80
6/6 [==============================] - 0s 9ms/step - loss: 17.6435 - mean_absolute_error: 3.5901 - val_loss: 60.5309 - val_mean_absolute_error: 7.5317
Epoch 7/80
6/

# <div style="color:white;display:fill;border-radius:20px;background-color:#f4ebdc;font-size:70%;font-family:Charter;letter-spacing:1px"><h3 style='padding: 20px;color:#8b4726;text-align:center;'>Model Testing</h3></div>

### Making Predictions

In [15]:
# Using model to predict sets
train_predictions = model.predict(X_train).flatten()
val_predictions = model.predict(X_val).flatten()
test_predictions = model.predict(X_test).flatten()

1/1 [==============================] - 0s 18ms/step


### Plotting Results

In [16]:
# Creating traces for all results
trace_y = go.Scatter(x=dates, y=y, name='Observations', opacity=0.5, line=dict(color='grey'))
trace_train_pred = go.Scatter(x=dates_train, y=train_predictions, name='Training Predictions')
trace_val_pred = go.Scatter(x=dates_val, y=val_predictions, name='Validation Predictions')
trace_test_pred = go.Scatter(x=dates_test, y=test_predictions, name='Testing Predictions')

# Combining traces, setting layout and plotting figure
data = [trace_y, trace_train_pred, trace_val_pred, trace_test_pred]
layout = go.Layout(title='Predictions Plot')
fig = go.Figure(data=data, layout=layout)
fig.show()

# <div style="color:white;display:fill;border-radius:20px;background-color:#f4ebdc;font-size:70%;font-family:Charter;letter-spacing:1px"><h3 style='padding: 20px;color:#8b4726;text-align:center;'>Summary</h3></div>

So, based on what we've seen, it seems like we're on the right track with using LSTM neural networks to predict future stock prices. Our exploratory data analysis helped us to gain insights into the C3.AI Stocks Dataset and understand the trends and patterns in the data. 
In preprocessing creating 3 lag features, splitting the data into training, validation, and testing sets was also a crucial step before building and training the LSTM model on the training set.

I'm quite impressed by the ability of the LSTM model to capture long-term dependencies in time-series data and make accurate predictions on future stock prices. This gives me confidence that we can use this knowledge to apply LSTM networks to other forecasting tasks as well.

Overall, I believe that this notebook provides a useful basic guide for applying LSTM networks to stock price prediction tasks and provides a foundation for further exploration and development in this area.